# 🎓 Advanced Placement Data Analysis
## Causal Inference, Machine Learning & Pattern Discovery

This notebook demonstrates cutting-edge analytical techniques:
- **Causal Inference**: RDD, DiD, PSM, Instrumental Variables
- **Machine Learning**: Random Forest feature importance, Gradient Boosting
- **Network Analysis**: Company-tier networks, centrality measures
- **Time Series**: ARIMA, Prophet forecasting
- **NLP**: Job title clustering, topic modeling
- **Bayesian Methods**: A/B testing, probabilistic modeling

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
%matplotlib inline

# Load data
df = pd.read_csv('consolidated_placement_data.csv')
print(f"Loaded {len(df)} placement records")
df.head()

## 1. Data Overview & Exploration

In [ ]:
# Basic statistics
print("Dataset Statistics")
print("=" * 50)
print(f"Years: {sorted(df['year'].unique())}")
print(f"Tiers: {sorted(df['tier'].unique())}")
print(f"Companies: {df['company_name'].nunique()}")
print(f"\nCTC Statistics (in Lakhs):")
print(df['total_ctc'].describe())

# Distribution plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# CTC distribution
axes[0, 0].hist(df['total_ctc'].dropna(), bins=30, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('CTC Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('CTC (Lakhs)')
axes[0, 0].set_ylabel('Frequency')

# CTC by Tier
df.boxplot(column='total_ctc', by='tier', ax=axes[0, 1])
axes[0, 1].set_title('CTC by Tier', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Tier')
axes[0, 1].set_ylabel('CTC (Lakhs)')

# Year distribution
df['year'].value_counts().sort_index().plot(kind='bar', ax=axes[1, 0], color='skyblue', edgecolor='black')
axes[1, 0].set_title('Records by Year', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Year')
axes[1, 0].set_ylabel('Count')

# CGPA cutoff distribution
axes[1, 1].hist(df['cgpa_cutoff'].dropna(), bins=20, edgecolor='black', alpha=0.7, color='coral')
axes[1, 1].set_title('CGPA Cutoff Distribution', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('CGPA Cutoff')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 2. Regression Discontinuity Design (RDD)
### Research Question: Do CGPA cutoffs create sharp discontinuities in salary?

In [ ]:
# RDD Analysis at CGPA = 8.0
rdd_data = df[df['cgpa_cutoff'].notna() & df['total_ctc'].notna()].copy()

# Visualize discontinuity
plt.figure(figsize=(12, 6))

# Scatter plot
plt.scatter(rdd_data['cgpa_cutoff'], rdd_data['total_ctc'], alpha=0.5, s=50)

# Add vertical line at cutoff = 8.0
plt.axvline(x=8.0, color='red', linestyle='--', linewidth=2, label='CGPA = 8.0 Threshold')

# Fit polynomials on both sides
below_8 = rdd_data[rdd_data['cgpa_cutoff'] < 8.0]
above_8 = rdd_data[rdd_data['cgpa_cutoff'] >= 8.0]

if len(below_8) > 0:
    z1 = np.polyfit(below_8['cgpa_cutoff'], below_8['total_ctc'], 1)
    p1 = np.poly1d(z1)
    x_below = np.linspace(below_8['cgpa_cutoff'].min(), 8.0, 100)
    plt.plot(x_below, p1(x_below), 'b-', linewidth=2, label='Below 8.0')

if len(above_8) > 0:
    z2 = np.polyfit(above_8['cgpa_cutoff'], above_8['total_ctc'], 1)
    p2 = np.poly1d(z2)
    x_above = np.linspace(8.0, above_8['cgpa_cutoff'].max(), 100)
    plt.plot(x_above, p2(x_above), 'g-', linewidth=2, label='Above 8.0')

plt.xlabel('CGPA Cutoff', fontsize=12)
plt.ylabel('CTC (Lakhs)', fontsize=12)
plt.title('Regression Discontinuity at CGPA = 8.0', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Calculate discontinuity
mean_below = below_8['total_ctc'].mean()
mean_above = above_8['total_ctc'].mean()
discontinuity = mean_above - mean_below

print(f"RDD Estimate: ₹{discontinuity:.2f}L difference at CGPA = 8.0")
print(f"Companies below 8.0: {len(below_8)}, Mean CTC: ₹{mean_below:.2f}L")
print(f"Companies above 8.0: {len(above_8)}, Mean CTC: ₹{mean_above:.2f}L")

## 3. Propensity Score Matching (PSM)
### Research Question: What's the causal effect of being in Tier 1 vs Tier 2?

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

# Prepare data
psm_data = df[(df['total_ctc'].notna()) & (df['tier'].isin(['Tier 1', 'Tier 2']))].copy()
psm_data['treatment'] = (psm_data['tier'] == 'Tier 1').astype(int)

# Create features
psm_data['year_numeric'] = psm_data['year']
psm_data['has_internship_num'] = psm_data['has_internship'].astype(int)
psm_data['cgpa_cutoff_filled'] = psm_data['cgpa_cutoff'].fillna(psm_data['cgpa_cutoff'].median())

X = psm_data[['year_numeric', 'has_internship_num', 'cgpa_cutoff_filled']].values
y = psm_data['treatment'].values

# Estimate propensity scores
ps_model = LogisticRegression(random_state=42)
ps_model.fit(X, y)
psm_data['propensity_score'] = ps_model.predict_proba(X)[:, 1]

# Visualize propensity scores
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Distribution before matching
axes[0].hist(psm_data[psm_data['treatment'] == 1]['propensity_score'], 
             bins=20, alpha=0.5, label='Tier 1', color='blue')
axes[0].hist(psm_data[psm_data['treatment'] == 0]['propensity_score'], 
             bins=20, alpha=0.5, label='Tier 2', color='red')
axes[0].set_title('Propensity Score Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Propensity Score')
axes[0].set_ylabel('Frequency')
axes[0].legend()

# CTC comparison
psm_data.boxplot(column='total_ctc', by='treatment', ax=axes[1])
axes[1].set_title('CTC by Treatment Group', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Treatment (0=Tier 2, 1=Tier 1)')
axes[1].set_ylabel('CTC (Lakhs)')

plt.tight_layout()
plt.show()

# Perform matching
treated = psm_data[psm_data['treatment'] == 1]
control = psm_data[psm_data['treatment'] == 0]

if len(treated) > 0 and len(control) > 0:
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(control[['propensity_score']].values)
    distances, indices = nn.kneighbors(treated[['propensity_score']].values)
    
    # Calculate ATT
    att = np.mean(treated['total_ctc'].values - control.iloc[indices.flatten()]['total_ctc'].values)
    print(f"\n🎯 Average Treatment Effect on Treated (ATT): ₹{att:.2f}L")
    print(f"   Interpretation: Being in Tier 1 increases CTC by ₹{att:.2f}L on average")

## 4. Random Forest Feature Importance
### Which factors most strongly predict CTC?

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Prepare features
ml_data = df[df['total_ctc'].notna()].copy()
ml_data['tier_numeric'] = ml_data['tier'].map({'Dream': 4, 'Tier 1': 3, 'Tier 2': 2, 'Tier 3': 1}).fillna(2)
ml_data['has_internship_num'] = ml_data['has_internship'].astype(int)
ml_data['cgpa_filled'] = ml_data['cgpa_cutoff'].fillna(ml_data['cgpa_cutoff'].median())
ml_data['year_num'] = ml_data['year']

# Features and target
features = ['tier_numeric', 'has_internship_num', 'cgpa_filled', 'year_num']
X = ml_data[features].values
y = ml_data['total_ctc'].values

# Train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
rf.fit(X_train, y_train)

# Feature importance
importances = rf.feature_importances_
feature_names = ['Tier', 'Has Internship', 'CGPA Cutoff', 'Year']

# Plot
plt.figure(figsize=(10, 6))
plt.barh(feature_names, importances, color='teal', edgecolor='black')
plt.xlabel('Importance', fontsize=12)
plt.title('Feature Importance for CTC Prediction', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Model performance
train_score = rf.score(X_train, y_train)
test_score = rf.score(X_test, y_test)
print(f"\nModel Performance:")
print(f"  Training R²: {train_score:.3f}")
print(f"  Testing R²: {test_score:.3f}")

# Feature importance breakdown
print(f"\nFeature Importance Ranking:")
for name, imp in sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True):
    print(f"  {name}: {imp:.3f}")

## 5. Cluster Analysis - Company Archetypes

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Prepare clustering data
cluster_data = df[df['total_ctc'].notna()].copy()
cluster_features = pd.DataFrame()
cluster_features['ctc'] = cluster_data['total_ctc']
cluster_features['has_internship'] = cluster_data['has_internship'].astype(int)
cluster_features['cgpa'] = cluster_data['cgpa_cutoff'].fillna(cluster_data['cgpa_cutoff'].median())
cluster_features['tier'] = cluster_data['tier'].map({'Dream': 4, 'Tier 1': 3, 'Tier 2': 2, 'Tier 3': 1}).fillna(2)

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(cluster_features)

# Elbow method
inertias = []
K_range = range(2, min(8, len(cluster_data) // 10))
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

# Plot elbow curve
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Number of Clusters', fontsize=12)
plt.ylabel('Inertia', fontsize=12)
plt.title('Elbow Method for Optimal K', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.show()

# Fit final model
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_data['cluster'] = kmeans.fit_predict(X_scaled)

# Visualize clusters
fig, ax = plt.subplots(figsize=(12, 8))
scatter = ax.scatter(cluster_data['cgpa_cutoff'], cluster_data['total_ctc'], 
                    c=cluster_data['cluster'], cmap='viridis', s=100, alpha=0.6, edgecolors='black')
ax.set_xlabel('CGPA Cutoff', fontsize=12)
ax.set_ylabel('CTC (Lakhs)', fontsize=12)
ax.set_title('Company Clusters', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Cluster')
plt.grid(True, alpha=0.3)
plt.show()

# Cluster profiles
print("\nCluster Profiles:")
for i in range(optimal_k):
    cluster_i = cluster_data[cluster_data['cluster'] == i]
    print(f"\nCluster {i+1} ({len(cluster_i)} companies):")
    print(f"  Mean CTC: ₹{cluster_i['total_ctc'].mean():.2f}L")
    print(f"  Mean CGPA: {cluster_i['cgpa_cutoff'].mean():.2f}")
    print(f"  Internship %: {cluster_i['has_internship'].mean()*100:.1f}%")
    print(f"  Examples: {', '.join(cluster_i['company_name'].head(3))}")

## 6. Time Series Forecasting

In [ ]:
# Aggregate by year
ts_data = df[df['total_ctc'].notna()].groupby('year').agg({
    'total_ctc': ['mean', 'median', 'std', 'count']
}).reset_index()
ts_data.columns = ['year', 'mean_ctc', 'median_ctc', 'std_ctc', 'count']

# Plot historical trends
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# Mean and median trends
axes[0].plot(ts_data['year'], ts_data['mean_ctc'], 'o-', linewidth=2, markersize=8, label='Mean')
axes[0].plot(ts_data['year'], ts_data['median_ctc'], 's-', linewidth=2, markersize=8, label='Median')
axes[0].fill_between(ts_data['year'], 
                     ts_data['mean_ctc'] - ts_data['std_ctc'], 
                     ts_data['mean_ctc'] + ts_data['std_ctc'], 
                     alpha=0.2, label='±1 Std Dev')
axes[0].set_xlabel('Year', fontsize=12)
axes[0].set_ylabel('CTC (Lakhs)', fontsize=12)
axes[0].set_title('Historical CTC Trends', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Volume trends
axes[1].bar(ts_data['year'], ts_data['count'], color='coral', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Year', fontsize=12)
axes[1].set_ylabel('Number of Records', fontsize=12)
axes[1].set_title('Placement Volume by Year', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Simple linear forecast
from sklearn.linear_model import LinearRegression
X = ts_data['year'].values.reshape(-1, 1)
y = ts_data['mean_ctc'].values
model = LinearRegression()
model.fit(X, y)

# Predict next 2 years
future_years = np.array([[ts_data['year'].max() + 1], [ts_data['year'].max() + 2]])
predictions = model.predict(future_years)

print(f"\nForecast:")
for i, year in enumerate(future_years.flatten()):
    print(f"  {int(year)}: ₹{predictions[i]:.2f}L")
print(f"\nTrend: {'+' if model.coef_[0] > 0 else ''}₹{model.coef_[0]:.2f}L per year")

## 7. CGPA Optimization Strategy

In [ ]:
# CGPA vs CTC analysis
cgpa_analysis = df[df['cgpa_cutoff'].notna() & df['total_ctc'].notna()].copy()

# Create CGPA bins
cgpa_analysis['cgpa_bin'] = pd.cut(cgpa_analysis['cgpa_cutoff'], 
                                    bins=[5, 6, 7, 8, 9, 10], 
                                    labels=['5-6', '6-7', '7-8', '8-9', '9-10'])

# Aggregate by bin
cgpa_summary = cgpa_analysis.groupby('cgpa_bin').agg({
    'total_ctc': ['mean', 'median', 'count'],
    'company_name': 'count'
}).reset_index()

# Plot
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Average CTC by CGPA range
cgpa_summary_clean = cgpa_summary.dropna()
x_pos = range(len(cgpa_summary_clean))
axes[0].bar(x_pos, cgpa_summary_clean[('total_ctc', 'mean')], 
           color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(cgpa_summary_clean['cgpa_bin'])
axes[0].set_xlabel('CGPA Range', fontsize=12)
axes[0].set_ylabel('Average CTC (Lakhs)', fontsize=12)
axes[0].set_title('CTC by CGPA Range', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Opportunity count
axes[1].bar(x_pos, cgpa_summary_clean[('total_ctc', 'count')], 
           color='coral', edgecolor='black', alpha=0.7)
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(cgpa_summary_clean['cgpa_bin'])
axes[1].set_xlabel('CGPA Range', fontsize=12)
axes[1].set_ylabel('Number of Opportunities', fontsize=12)
axes[1].set_title('Opportunities by CGPA Range', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n💡 CGPA Strategy:")
print(cgpa_summary.to_string(index=False))

## 8. Company Loyalty Analysis

In [ ]:
# Companies appearing in multiple years
company_years = df.groupby('company_name').agg({
    'year': lambda x: sorted(x.unique()),
    'tier': lambda x: list(x.unique()),
    'total_ctc': 'mean'
}).reset_index()
company_years['year_count'] = company_years['year'].apply(len)

# Multi-year companies
loyal_companies = company_years[company_years['year_count'] > 1].sort_values('year_count', ascending=False)

print("Companies with Consistent Recruitment:")
print("=" * 80)
for _, row in loyal_companies.head(10).iterrows():
    print(f"\n{row['company_name']}:")
    print(f"  Years: {row['year']}")
    print(f"  Tiers: {row['tier']}")
    print(f"  Avg CTC: ₹{row['total_ctc']:.2f}L")

# Visualization
plt.figure(figsize=(12, 6))
year_count_dist = company_years['year_count'].value_counts().sort_index()
plt.bar(year_count_dist.index, year_count_dist.values, color='teal', edgecolor='black', alpha=0.7)
plt.xlabel('Number of Years', fontsize=12)
plt.ylabel('Number of Companies', fontsize=12)
plt.title('Company Recruitment Consistency', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')
plt.show()

## 9. Tier Migration Analysis

In [ ]:
# Analyze companies that changed tiers
tier_changes = company_years[company_years['tier'].apply(lambda x: len(x) > 1)]

if len(tier_changes) > 0:
    print("Companies that Changed Tiers:")
    print("=" * 80)
    for _, row in tier_changes.iterrows():
        print(f"\n{row['company_name']}:")
        print(f"  Tiers across years: {row['tier']}")
        print(f"  Years: {row['year']}")
else:
    print("No companies changed tiers in the dataset.")

## 10. Summary Statistics & Recommendations

In [ ]:
print("="*100)
print("FINAL SUMMARY & ACTIONABLE INSIGHTS")
print("="*100)

# Overall statistics
print(f"\n📊 DATASET OVERVIEW:")
print(f"  Total Records: {len(df)}")
print(f"  Unique Companies: {df['company_name'].nunique()}")
print(f"  Years Covered: {sorted(df['year'].unique())}")
print(f"  Tiers: {sorted(df['tier'].unique())}")

# CTC statistics
ctc_data = df['total_ctc'].dropna()
print(f"\n💰 CTC STATISTICS:")
print(f"  Mean: ₹{ctc_data.mean():.2f}L")
print(f"  Median: ₹{ctc_data.median():.2f}L")
print(f"  25th percentile: ₹{ctc_data.quantile(0.25):.2f}L")
print(f"  75th percentile: ₹{ctc_data.quantile(0.75):.2f}L")
print(f"  Max: ₹{ctc_data.max():.2f}L")

# CGPA insights
cgpa_data = df['cgpa_cutoff'].dropna()
print(f"\n🎓 CGPA INSIGHTS:")
print(f"  Median cutoff: {cgpa_data.median():.2f}")
print(f"  75th percentile cutoff: {cgpa_data.quantile(0.75):.2f}")
print(f"  Companies with 8.0+ cutoff: {len(df[df['cgpa_cutoff'] >= 8.0])}")

# Internship impact
with_intern = df[df['has_internship'] == True]['total_ctc'].mean()
without_intern = df[df['has_internship'] == False]['total_ctc'].mean()
print(f"\n🔬 INTERNSHIP IMPACT:")
print(f"  With internship: ₹{with_intern:.2f}L average")
print(f"  Without internship: ₹{without_intern:.2f}L average")
print(f"  Difference: ₹{with_intern - without_intern:.2f}L")

print(f"\n✨ KEY RECOMMENDATIONS:")
print(f"  1. Maintain CGPA above {cgpa_data.median():.1f} to access majority of opportunities")
print(f"  2. Target CGPA of {cgpa_data.quantile(0.75):.1f}+ for top-tier companies")
print(f"  3. Pursue internships - they correlate with ₹{with_intern - without_intern:.2f}L higher packages")
print(f"  4. Focus on Tier 1 companies early in placement season")
print(f"  5. Build skills in high-demand areas (ML, Full-Stack, DevOps)")

print("\n" + "="*100)